### Importing libraries



In [ ]:
!pip install kaggle
!pip install zip
!pip install unidecode
import json
import os, sys, stat
import shutil
import pandas as pd
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import RSLPStemmer
nltk.download('rslp')
stemmer = RSLPStemmer()
stopwords = nltk.corpus.stopwords.words('english')

#ML Pre processing

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


### Importing dataset

Kaggle class allow you to connect to kaggle and retrieve your dataset by providing your credentials json variable "kj_object".

dumpJson() - Creates a folder path with json for kaggle API auth

get_dataset() - Retrieves your kaggle dataset

In [ ]:
kj_object = {
      "username":"<kaggle_username>",
      "key":"<kaggle_api_token>"
  }

class Kaggle():

    def __init__(self, kj_object: object):
      self.kj_object = kj_object

    def dumpJson(self):

    #Kaggle API Token

    #Validating if path already exists

      if os.path.exists('/root/.kaggle'):
        shutil.rmtree('/root/.kaggle')
        print("--> Deleting already existing 'Kaggle' repository")
 
      print("--> Creating new file at .kaggle")
      os.makedirs('../root/.kaggle', exist_ok = True)

      #shutil.move('.kaggle', '/root/')
      

      kaggle_Json = json.dumps(self.kj_object)
      kaggle_json_file = open('/root/.kaggle/kaggle.json', 'w')
      kaggle_json_file.write(kaggle_Json)
      kaggle_json_file.close()

      #!mv /root/.kaggle /content/
      os.chmod('/root/.kaggle/kaggle.json', 600) 

    def get_dataset(self):

      !kaggle datasets download -d reihanenamdari/mental-health-corpus


In [ ]:
kaggle_conn = Kaggle(kj_object)

In [ ]:
kaggle_conn.dumpJson()

--> Creating new file at .kaggle


In [ ]:
kaggle_conn.get_dataset()

100% 4.74M/4.74M [00:00<00:00, 38.8MB/s]
100% 4.74M/4.74M [00:00<00:00, 38.6MB/s]


In [ ]:
def unzipDataset(path_from, path_target):

  import zipfile
  with zipfile.ZipFile(path_from, 'r') as zip_ref:
    zip_ref.extractall(path_target)

  os.remove(path_from)

  return 

In [ ]:
unzipDataset('/content/mental-health-corpus.zip', '/content/sample_data')

In [ ]:
df = pd.read_csv('/content/sample_data/mental_health.csv')

In [ ]:
df.head(100)

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1
...,...,...
95,weird or im ambivert already tell pretty weird...,0
96,weeks ago saved friend redflag suspended reddi...,0
97,emo anything but songs lonely palaye royal iro...,0
98,else dobefore begin long post amp hope take ti...,1


### Preparing data

The class data_normalization described bellow allow you to apply some methods for preparing your dataset inputs such as: removing special characters and removing extra spaces, reducing each word to its root (word stemming) and removing stopwords.

All the operations are applied on tokenized strings.

In [ ]:
class data_normalization():

  def __init__(self, dataframe):
    self.dataframe = df
    

  def normalize_spaces(df):
    df.text_norm = [re.sub(r"[^a-zA-Z0-9 ]","",text) for text in df.text.values]
    df.text_norm = [re.sub(r"\s+"," ",text) for text in df.text.values]

  
  def stemm_words(df):
    stemmed_tokens = []
    for sentence in df.text:
        stemmed_tokens.append(" ".join([stemmer.stem(i) for i in sentence.split()]))
    # print(len(stemmed_tokens))
    return stemmed_tokens


  def remove_stopwords(df):

    tokens_wtout_sw = []

    for sentence in df:
      sentence = ' '.join([word for word in sentence.split() if word not in stopwords])
      tokens_wtout_sw.append(sentence)

    return tokens_wtout_sw


In [ ]:
stemmed_tokens = data_normalization.stemm_words(df)

In [ ]:
df_text_tokens = data_normalization.remove_stopwords(stemmed_tokens)

In [ ]:
#Making sure that stopwords were removed

count_before_stopwords=0
for sentence in df.text.values:
  for word in sentence.split(' '):
    count_before_stopwords+=1

print(f'Before removing stopwords: {count_before_stopwords}')


count_after_stopwords=0
for sentence in df_text_tokens:
  for word in sentence.split(' '):
      count_after_stopwords+=1

print(f'After removing stopwords: {count_after_stopwords}')


Before removing stopwords: 2043667
After removing stopwords: 1905141


In [ ]:
df_text_tokens[:5]

['de american te question dutch person heard guy get way easi thing learn age us soooo thth grad lik right guy learn math',
 'nothing look forward lif dont many reasom keep going feel lik nothing keep going next day mak want hang',
 'music recommendatiom im looking expand playlist usual genr alt pop minnesot hip hop steampunk vari indi genr artist peopl lik cavetown aliceband bug hunt penelop scott vari rhymesay willing explor new genresartist anything generic rap typ exclusively sex drug cool rapp rap typ pretty good pop popul coupl ye ago dunn technic genr nam anyway anyon got music recommendatiom favorit artistssong',
 'im trying feel betterth reason im still aliv know mum devastated ev killed ev pass im still stat im going hesitat ending lif shortly aft im almost tak med go therapy nothing se help enough dont want around anymor hat feeling lik thil wouldnt wish upon enemy brain feel lik constantly lik static tv wont shut overthinking think im running optiom dont see living past got

### Enconding targets

Labels are already encoded on dataset, but to make sure it's not going to fail let's enforce it :)

In [ ]:
lenc = preprocessing.LabelEncoder()

In [ ]:
lenc_target = lenc.fit(df.label)

In [ ]:
list(lenc_target.classes_)

[0, 1]

### Splitting data and Feature extraction

In [ ]:
vec = TfidfVectorizer(max_features=3000)

X_train, X_test, Y_train, Y_test = train_test_split(vec.fit_transform(df_text_tokens), df.label.values, test_size=0.2, random_state=42)

### Model fitting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

model_rf= RandomForestClassifier(n_estimators=10,random_state=42)

In [ ]:
predict = model_rf.fit(X_train, Y_train)

### Model metrics

In [ ]:
print("Train Accuracy  : {:.2f} %".format(accuracy_score(model_rf.predict(X_train), Y_train)*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(model_rf.predict(X_test), Y_test)*100))
print("Precision       : {:.2f} %".format(precision_score(model_rf.predict(X_test), Y_test,average='macro')*100))
print("Recall          : {:.2f} %".format(recall_score(model_rf.predict(X_test), Y_test,average='macro')*100))

Train Accuracy  : 99.35 %
Test Accuracy   : 87.53 %
Precision       : 87.53 %
Recall          : 87.53 %


### Testing own string

In [ ]:
dict_labels = {0:'Possible Mental Health Issues - NOT IDENTIFIED', 1:'Possible Mental Health Issues - IDENTIFIED'}

Down bellow you can type your own strings on list "new_input" to test the model :)

In [ ]:
new_input = ['today im feeling like dont want to live... dont know why but i see no reason for being here. Good bye forever ill suicide', 'woke up in an awesome mood today', 'her atitude gave me anxiety and a deep sadness... dont want to live it again and dont even look at her face once more']


In [ ]:
def normalize_string(inputData):

  #Normalize spaces and remove specials
  inputData = [re.sub(r"[^a-zA-Z0-9 ]","",text) for text in inputData]
  inputData = [re.sub(r"\s+"," ",text) for text in inputData]

  # #Tokenize, stem and remove stopwords
  tokenized_strings = []
  for text in inputData:
    tokenized_words = [stemmer.stem(word) for word in text.lower().split() if word not in stopwords]
    tokenized_string = " ".join(tokenized_words)
    tokenized_strings.append(tokenized_string)

  return tokenized_strings

In [ ]:
tokenized_strings = normalize_string(new_input)

In [ ]:
tokenized_strings

['today im feeling lik dont want liv dont know see reason good bye forev ill suicid',
 'wok awesom mood today',
 'atitud gav anxiety deep sadnes dont want liv dont even look fac']

In [ ]:
new_vec = vec.fit(df_text_tokens)

In [ ]:
new_vec_transformed = vec.transform(tokenized_strings)

In [ ]:
new_pred = model_rf.predict(new_vec_transformed)

In [ ]:
new_pred_proba = model_rf.predict_proba(new_vec_transformed)

In [ ]:
count = 0
for result in range (0, len(new_pred)):
  proba = new_pred_proba[count][new_pred[result]]*100
  print(f'Data: {tokenized_strings[result]}\nResult: {dict_labels[new_pred[result]]}\nConfidence:{proba: .2f}%\n\n')
  count+=1

Data: today im feeling lik dont want liv dont know see reason good bye forev ill suicid
Result: Possible Mental Health Issues - IDENTIFIED
Confidence: 60.00%


Data: wok awesom mood today
Result: Possible Mental Health Issues - NOT IDENTIFIED
Confidence: 100.00%


Data: atitud gav anxiety deep sadnes dont want liv dont even look fac
Result: Possible Mental Health Issues - IDENTIFIED
Confidence: 60.00%


